In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

### 首先我们先看看我们的数据集cnews.test.txt, cnews.train.txt, cnews.val.txt的文本内容是什么样的

文本中一行为一个样本，格式为："主题\t内容"

In [2]:
# 测试集数据
test_data = pd.read_csv('./cnews.test.txt',sep='\t',engine='python',names=['label','content'],encoding='UTF-8')
print(test_data.shape)
print(test_data.head())

(10000, 2)
  label                                            content
0    体育  鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今，本赛季的NCAA进入到了末...
1    体育  麦基砍28+18+5却充满寂寞 纪录之夜他的痛阿联最懂新浪体育讯上天对每个人都是公平的，贾维...
2    体育  黄蜂vs湖人首发：科比冲击七连胜 火箭两旧将登场新浪体育讯北京时间3月28日，NBA常规赛洛...
3    体育  双面谢亚龙作秀终成做作 谁来为低劣行政能力埋单是谁任命了谢亚龙？谁放纵了谢亚龙？谁又该为谢亚...
4    体育  兔年首战山西换帅后有虎胆 张学文用乔丹名言励志今晚客场挑战浙江稠州银行队，是山西汾酒男篮的兔...


In [6]:
# 训练集数据
train_data = pd.read_csv('./cnews.train.txt',sep='\t',engine='python',names=['label','content'],encoding='UTF-8')
print(train_data.shape)
train_data.head()

(50000, 2)


,label,content
0,体育,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1,体育,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2,体育,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3,体育,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4,体育,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...


In [7]:
# 验证集数据
validation_data = pd.read_csv('./cnews.val.txt',sep='\t',engine='python',names=['label','content'],encoding='UTF-8')
print(test_data.shape)
validation_data.head()

(10000, 2)


,label,content
0,体育,黄蜂vs湖人首发：科比带伤战保罗 加索尔救赎之战 新浪体育讯北京时间4月27日，NBA季后赛...
1,体育,1.7秒神之一击救马刺王朝于危难 这个新秀有点牛！新浪体育讯在刚刚结束的比赛中，回到主场的马...
2,体育,1人灭掘金！神般杜兰特！ 他想要分的时候没人能挡新浪体育讯在NBA的世界里，真的猛男，敢于直...
3,体育,韩国国奥20人名单：朴周永领衔 两世界杯国脚入选新浪体育讯据韩联社首尔9月17日电 韩国国奥...
4,体育,天才中锋崇拜王治郅 周琦：球员最终是靠实力说话2月14日从土耳其男篮邀请赛回到北京之后，周琦...


#### 查看下训练集总共有多少种类别，以及各类别的数量

In [14]:
s = train_data['label'].value_counts()
print(s)

时政    5000
科技    5000
游戏    5000
财经    5000
时尚    5000
房产    5000
教育    5000
体育    5000
家居    5000
娱乐    5000
Name: label, dtype: int64


In [23]:
label2id_dict = {x:i for i,x in enumerate(train_data['label'].value_counts().index)}
print(label2id)

{'时政': 0, '科技': 1, '游戏': 2, '财经': 3, '时尚': 4, '房产': 5, '教育': 6, '体育': 7, '家居': 8, '娱乐': 9}


## 数据预处理

### 1、通过训练集数据，建立我们自己的字典集

In [39]:
from collections import Counter
def build_voca_txt(train_dir, save_path, vocab_size=5000):
    """
    创建字典文件
    """
    train_data = pd.read_csv(train_dir,sep='\t',engine='python',names=['label','content'],encoding='UTF-8')
    df = train_data['content'].apply(lambda x:[word for word in x],1)
    print(df.head())
    
    all_data = []
    for line in df:
        all_data.extend(line)
    
    #学习Counter类的使用，all_data为一维列表
    counter = Counter(all_data)
    #去出现次数最多前5000-1个
    count_pairs = counter.most_common(vocab_size - 1)
    #只取字符，不取个数
    words, _ = list(zip(*count_pairs))
    #添加<PAD>标识符
    words = ['<PAD>'] + list(words)
    #保存到文件里
    open(save_path, mode='w',encoding='utf-8').write('\n'.join(words) + '\n')

build_voca_txt('./cnews.train.txt','./vocab.txt')

0    [马, 晓, 旭, 意, 外, 受, 伤, 让, 国, 奥, 警, 惕,  , 无, 奈, ...
1    [商, 瑞, 华, 首, 战, 复, 仇, 心, 切,  , 中, 国, 玫, 瑰, 要, ...
2    [冠, 军, 球, 队, 迎, 新, 欢, 乐, 派, 对,  , 黄, 旭, 获, 大, ...
3    [辽, 足, 签, 约, 危, 机, 引, 注, 册, 难, 关,  , 高, 层, 威, ...
4    [揭, 秘, 谢, 亚, 龙, 被, 带, 走, ：, 总, 局, 电, 话, 骗, 局, ...
Name: content, dtype: object


### 2、读取字典的方法，供后续使用

In [20]:
def read_vocab(vocab_path):
    """
    读取字典，保存格式：{字符：序号}
    """
    vocab_dict = {}
    with open(vocab_path,encoding='utf-8') as f:
        for i,word in enumerate(f):
            vocab_dict[word[:-1]] = i
    return vocab_dict

vocab_dict = read_vocab('./vocab.txt')
print(len(vocab_dict))
print(vocab_dict.items())

5000
dict_items([('<PAD>', 0), ('，', 1), ('的', 2), ('。', 3), ('一', 4), ('是', 5), ('在', 6), ('0', 7), ('有', 8), ('不', 9), ('了', 10), ('中', 11), ('1', 12), ('人', 13), ('大', 14), ('、', 15), ('国', 16), (' ', 17), ('2', 18), ('这', 19), ('上', 20), ('为', 21), ('个', 22), ('“', 23), ('”', 24), ('年', 25), ('学', 26), ('时', 27), ('我', 28), ('地', 29), ('和', 30), ('以', 31), ('到', 32), ('出', 33), ('来', 34), ('会', 35), ('行', 36), ('发', 37), ('：', 38), ('对', 39), ('们', 40), ('要', 41), ('生', 42), ('家', 43), ('他', 44), ('能', 45), ('也', 46), ('业', 47), ('金', 48), ('3', 49), ('成', 50), ('可', 51), ('分', 52), ('多', 53), ('现', 54), ('5', 55), ('就', 56), ('场', 57), ('新', 58), ('后', 59), ('于', 60), ('下', 61), ('日', 62), ('经', 63), ('市', 64), ('前', 65), ('过', 66), ('方', 67), ('得', 68), ('作', 69), ('月', 70), ('最', 71), ('开', 72), ('房', 73), ('》', 74), ('《', 75), ('高', 76), ('9', 77), ('8', 78), ('.', 79), ('而', 80), ('比', 81), ('公', 82), ('4', 83), ('说', 84), (')', 85), ('将', 86), ('(', 87), ('都', 88), ('资', 89),

In [50]:
def idtext_decode(idtext, vocab_dict):
    """
    将id文本转换为字符文本
    """
    return ' '.join([vocab_dict.get(i,'?') for i in idtext])

In [35]:
def process_text(filename,vocab_path,max_length=1000):
    """
    
    """
    file_data = pd.read_csv(filename,sep='\t',engine='python',names=['label','content'],encoding='UTF-8')
    
    vocab_id = read_vocab(vocab_path)
    
    y = file_data['label'].map(label2id_dict)
    X=[]
    
    for words in file_data['content']:
#         将字符文本转换为id文本
        id_list = [vocab_id[x] for x in words if x in vocab_id]
        
#         将文本pad为固定长度
        id_list_length = len(id_list)
        if id_list_length <= max_length:
            id_list.extend([0]*(max_length - id_list_length)) # PAD
        else:
            id_list = id_list[:max_length]
        X.append(id_list)

    return np.array(X),np.array(y)

In [36]:
process_text('./cnews.train.txt','./vocab.txt')

(array([[ 387, 1197, 2173, ...,    0,    0,    0],
        [ 199,  964,  280, ...,   14,    2,  101],
        [1108,  581,  157, ...,  709,    2,  381],
        ...,
        [1687,   92,  397, ...,  163,   89,  135],
        [1687,   92,  397, ...,    0,    0,    0],
        [ 779,   25,  193, ...,   72,  358,  253]]),
 array([7, 7, 7, ..., 3, 3, 3], dtype=int64))